In [ ]:
import configparser
from datetime import datetime, timedelta

import numpy as np
from bokeh.plotting import show

import ssutils

In [ ]:
config = configparser.ConfigParser()
config.read('/run/secrets/db_config')

su = ssutils.SsMongo(config['db']['host'], config['db']['username'], config['db']['password'], int(config['db']['port']))
su.now = datetime.now()

In [ ]:
total_user_count = su.find_user_count()
active_user_count_week = su.find_user_count(activeperiod_days=7)
recent_activity_counts = su.find_event_count('activities', 1)
week_activity_counts = su.find_event_count('activities', 7)
recent_heartbeat_counts = su.find_event_count('heartbeats', 1)
week_heartbeat_counts = su.find_event_count('heartbeats', 7)
recent_screen_counts = su.find_event_count('sleeps', 1)
week_screen_counts = su.find_event_count('sleeps', 7)
week_attendee_log_counts = su.find_event_count('attendeelogs', 7)
weekly_attendee_log_inactive_users = su.find_inactive_attendeelog_user_count(5)
isolated_sleeps_per_user, users_with_isolated_sleep_count, users_with_isolated_sleeps = su.find_isolated_sleeps(1)
proportions, ratios, proportion_users, ratio_users = su.find_sleep_imbalance(1, threshold_proportion=0.5, threshold_ratio=0.5)
recent_rest_counts, _ = su.find_rest_null_count(1)
recent_rest_counts_threshold, recent_rests_users_threshold = su.find_rest_null_count(1, 1)

inactive_attendeelog_users = su.find_inactive_event_users('attendeelogs', 7)
inactive_attendeelog_phone_details = su.get_phone_details(inactive_attendeelog_users)
inactive_activity_users = su.find_inactive_event_users('activities', 7)
inactive_activity_phone_details = su.get_phone_details(inactive_activity_users)
inactive_heartbeat_users = su.find_inactive_event_users('heartbeats', 7)
inactive_heartbeat_phone_details = su.get_phone_details(inactive_heartbeat_users)
isolated_sleep_phone_details = su.get_phone_details(users_with_isolated_sleeps)
proportion_phone_details = su.get_phone_details(proportion_users)
ratio_phone_details = su.get_phone_details(ratio_users)
rests_threshold_phone_details = su.get_phone_details(recent_rests_users_threshold)

## Weekly report
### Sessions
**Total users**: _Number of users which have at least one record in at least one of the following collections: activities, sleeps, heartbeats_

**Active users within the last week**: _Number of users which have at least one record during last week in at least one of the following collections: activities, sleeps, heartbeats_

In [ ]:
print(f'{active_user_count_week} active users within the last week of {total_user_count} total users in the system')

### Activities

In [ ]:
# Activity — Last 24h statistics
hist, edges = np.histogram(recent_activity_counts, density=False, bins=100)
if len(recent_activity_counts) != 0:
    print('Min activity count last 24h for users with activities last 24h:', min(recent_activity_counts))
    print('Max activity count last 24h for users with activities last 24h:', max(recent_activity_counts))
    print('Median activity count last 24h for users with activities last 24h:', int(np.median(recent_activity_counts)))
print('Total users with activities last 24h:', len(recent_activity_counts))
print('Number of active users within the last week with no activities within the last week:', active_user_count_week-len(week_activity_counts))
show(ssutils.make_histogram_plot("activity", hist, edges))

### Heartbeats
**Isolated sleep event count**: _Number of screen events registered without heartbeats per user, i.e. a distance of minimum 30 minutes from a screen event to a heartbeat_

In [ ]:
# Heartbeats — Last 24h statistics
hist, edges = np.histogram(recent_heartbeat_counts, density=False, bins=100)
if len(recent_heartbeat_counts) != 0:
    print('Min heartbeat count last 24h for users with heartbeats last 24h:', min(recent_heartbeat_counts))
    print('Max heartbeat count last 24h for users with heartbeats last 24h:', max(recent_heartbeat_counts))
    print('Median heartbeat count last 24h for users with heartbeats last 24h:', int(np.median(recent_heartbeat_counts)))
print('Total users with heartbeats last 24h:', len(recent_heartbeat_counts))
print('Number of active users within the last week without heartbeats within the last week:', active_user_count_week-len(week_heartbeat_counts))
print('Number of users with isolated sleep events last 24h:', users_with_isolated_sleep_count)
show(ssutils.make_histogram_plot("heartbeat", hist, edges))
hist, edges = np.histogram(isolated_sleeps_per_user, density=False, bins=100)
show(ssutils.make_histogram_plot("isolated sleep event", hist, edges))

### Screen activation (sleeps)

In [ ]:
# Screens — Last 24h statistics
hist, edges = np.histogram(recent_screen_counts, density=False, bins=100)
if len(recent_screen_counts) != 0:
    print('Min screen count last 24h for users with sleeps last 24h:', min(recent_screen_counts))
    print('Max screen count last 24h for users with sleeps last 24h:', max(recent_screen_counts))
    print('Median screen count last 24h for users with sleeps last 24h:', int(np.median(recent_screen_counts)))
print('Total users with sleeps last 24h:', len(recent_screen_counts))
print('Number of active users within the last week without sleeps within the last week:', active_user_count_week-len(week_screen_counts))
show(ssutils.make_histogram_plot("screen", hist, edges))

#### Proportion of sequences with several following screen on or screen off events per user last 24 hours
**It is calculated as following**: _Number of two events of the same type following one another (on-on or off-off) is divided by number of screen events minus one.  So the resulting proportion could be in range [0, 1].  Proportion of 0 means there are no screen on events followed by screen on and there are no screen off events followed by screen off.  Proportion of 1 means that all events are of the same type and number of events is > 1._

In [ ]:
hist, edges = np.histogram(proportions, density=False, bins=100)
show(ssutils.make_histogram_plot("several on/off proportion", hist, edges, "several on/off proportion"))

#### Ratio for screen on and screen off per user last 24 hours
_The ratio is calculated so that it is always within [0, 1]. If number of screen on and screen off events is the same then ratio is 1.  If there are only screen events of one type then the ratio is 0.  If there are more screen on events then the ratio is (number of screen off events / number of screen on events), otherwise the ratio is (number of screen on events / number of screen off events)._

In [ ]:
hist, edges = np.histogram(ratios, density=False, bins=100)
show(ssutils.make_histogram_plot("on/off ratio", hist, edges, "on/off ratio"))

### Attendee logs

In [ ]:
# Attendee logs — Last week statistics
hist, edges = np.histogram(week_attendee_log_counts, density=False, bins=100)
if len(week_attendee_log_counts) != 0:
    print('Min number of changeddatetime last week for users with attendeelogs last week:', min(week_attendee_log_counts))
    print('Max number of changeddatetime last week for users with attendeelogs last week:', max(week_attendee_log_counts))
    print('Median number of changeddatetime last week for users with attendeelogs last week:', int(np.median(week_attendee_log_counts)))
print('Total users with attendeelogs last week:', len(week_attendee_log_counts))
print('Total users with number of chandeddatetime > 1 last week:', (np.asarray(week_attendee_log_counts)>1).sum())
show(ssutils.make_histogram_plot("attendee log change", hist, edges))

_The following bar plot shows 5 weeks, each bar on x axis shows number of users without chandeddatetime changes for that specific week. So on y axis is usercount and x axis is weeknumber. Weeknumbers can be -4,-3,-2,-1,0 (4 weeks ago, 3 weeks ago etc). Sum of those bars is not the total number of users which have chandeddatetime changes, since some users may have several weeks without change, so it'll be counted several times._

In [ ]:
# Attendee logs — inactive users by week
show(ssutils.make_histogram_plot("Inactive attendeelog by week", weekly_attendee_log_inactive_users, [-i for i in range(len(weekly_attendee_log_inactive_users)+1)], 'weeks'))

### Rests

In [ ]:
if len(recent_rest_counts) != 0:
    print('Min number of restperiods where endTime = null per user during last 24 hours:', min(recent_rest_counts))
    print('Max number of restperiods where endTime = null per user during last 24 hours:', max(recent_rest_counts))
    print('Median number of restperiods where endTime = null per user during last 24 hours:', int(np.median(recent_rest_counts)))
print('Total users with restperiods where endTime = null per user during last 24 hours:', len(recent_rest_counts))
print('Number of users with restperiods where endTime = null larger than 1 during last 24 hours:', len(recent_rest_counts_threshold))
hist, edges = np.histogram(recent_rest_counts, density=False, bins=100)
show(ssutils.make_histogram_plot("restperiods where endTime = null per user during last 24 hours", hist, edges))

### Phone and software

#### Active users with no activities within the last week

In [ ]:
summary, result = su.expand_phone_details(inactive_activity_phone_details)
print('Summary:\n')
ssutils.print_dict(summary)
print('\nPer-user details\n')
ssutils.print_dict(result)

#### Active users without heartbeats within the last week

In [ ]:
summary, result = su.expand_phone_details(inactive_heartbeat_phone_details)
print('Summary:\n')
ssutils.print_dict(summary)
print('\nPer-user details\n')
ssutils.print_dict(result)

#### Users with sleep events without heartbeats, i.e. a distance of minimum 30 minutes from a screen event to a heartbeat, during last day

In [ ]:
summary, result = su.expand_phone_details(isolated_sleep_phone_details)
print('Summary:\n')
ssutils.print_dict(summary)
print('\nPer-user details\n')
ssutils.print_dict(result)

#### Users with a proportion of several following screen on or of events larger than 0.5, during last day

In [ ]:
summary, result = su.expand_phone_details(proportion_phone_details)
print('Summary:\n')
ssutils.print_dict(summary)
print('\nPer-user details\n')
ssutils.print_dict(result)

#### Users with a ratio of screen on/screen of ratio smaller than 0.5, during last day

In [ ]:
summary, result = su.expand_phone_details(ratio_phone_details)
print('Summary:\n')
ssutils.print_dict(summary)
print('\nPer-user details\n')
ssutils.print_dict(result)

#### Active users without changes in bedtimes during last week

In [ ]:
summary, result = su.expand_phone_details(inactive_attendeelog_phone_details)
print('Summary:\n')
ssutils.print_dict(summary)
print('\nPer-user details\n')
ssutils.print_dict(result)

#### Users with restperiods where endTime = null per 24 hour > larger than 1

In [ ]:
summary, result = su.expand_phone_details(rests_threshold_phone_details)
print('Summary:\n')
ssutils.print_dict(summary)
print('\nPer-user details\n')
ssutils.print_dict(result)